<a href="https://colab.research.google.com/github/Abubakar17/Depth-Anything/blob/main/Copy_of_d.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install supervision
!pip install ultralytics

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.7/86.7 kB 2.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 716.0/716.0 kB 9.3 MB/s eta 0:00:00


In [3]:
#for sea-thru
!pip install rawpy"

/bin/bash: -c: line 1: unexpected EOF while looking for matching `"'
/bin/bash: -c: line 2: syntax error: unexpected end of file


In [4]:
!git clone https://github.com/Abubakar17/Depth-Anything/
%cd Depth-Anything
!pip install -r requirements.txt
!pip install --upgrade timm
!pip install open3d

Cloning into 'Depth-Anything'...
remote: Enumerating objects: 413, done.
remote: Counting objects: 100% (140/140), done.
remote: Compressing objects: 100% (113/113), done.
remote: Total 413 (delta 78), reused 71 (delta 27), pack-reused 273
Receiving objects: 100% (413/413), 237.88 MiB | 28.84 MiB/s, done.
Resolving deltas: 100% (135/135), done.
Updating files: 100% (219/219), done.
/content/Depth-Anything
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 97.4/97.4 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.6/16.6 MB 15.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.1/92.1 kB 11.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 305.1/305.1 kB 19.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 138.5/138.5 kB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 7.7

In [5]:
%cd /content/Depth-Anything/metric_depth/
%mkdir /content/Depth-Anything/metric_depth/checkpoints/

# Use wget to download the file
!wget "https://huggingface.co/spaces/LiheYoung/Depth-Anything/resolve/main/checkpoints_metric_depth/depth_anything_metric_depth_outdoor.pt?download=true" -O depth_anything_metric_depth_outdoor.pt
!wget 'https://huggingface.co/spaces/LiheYoung/Depth-Anything/resolve/main/checkpoints/depth_anything_vitl14.pth'


import shutil
import os
# Source directory path
source_directory = '/content/Depth-Anything/metric_depth/'

# Destination directory path
destination_directory = '/content/Depth-Anything/metric_depth/checkpoints/'

# Get a list of all files in the source directory
files = ['depth_anything_vitl14.pth' , 'depth_anything_metric_depth_outdoor.pt']

# Move each file individually
for file in files:
    source_path = os.path.join(source_directory, file)
    destination_path = os.path.join(destination_directory, file)
    shutil.move(source_path, destination_path)


/content/Depth-Anything/metric_depth
--2024-02-24 17:46:44--  https://huggingface.co/spaces/LiheYoung/Depth-Anything/resolve/main/checkpoints_metric_depth/depth_anything_metric_depth_outdoor.pt?download=true
Resolving huggingface.co (huggingface.co)... 108.138.94.45, 108.138.94.27, 108.138.94.52, ...
Connecting to huggingface.co (huggingface.co)|108.138.94.45|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://cdn-lfs-us-1.huggingface.co/repos/b2/a8/b2a84b9a6ef705fba73e7ccec6a9728b3427d8b4c7f536eae186110f0cbd700f/50a33976a4c909bd2284395d686bd644ce06c5c1c70f870d032b0eee93bdc56e?response-content-disposition=attachment%3B+filename*%3DUTF-8%27%27depth_anything_metric_depth_outdoor.pt%3B+filename%3D%22depth_anything_metric_depth_outdoor.pt%22%3B&Expires=1709056005&Policy=eyJTdGF0ZW1lbnQiOlt7IkNvbmRpdGlvbiI6eyJEYXRlTGVzc1RoYW4iOnsiQVdTOkVwb2NoVGltZSI6MTcwOTA1NjAwNX19LCJSZXNvdXJjZSI6Imh0dHBzOi8vY2RuLWxmcy11cy0xLmh1Z2dpbmdmYWNlLmNvL3JlcG9zL2IyL2E4L2IyYTg0Yjlh

In [6]:
# infer_depth_video.py
# Modified by @ChatGPT
%cd /content/Depth-Anything/metric_depth
import argparse
import os
import torch
import numpy as np
from PIL import Image
import torchvision.transforms as transforms
from tqdm import tqdm
from zoedepth.models.builder import build_model
from zoedepth.utils.config import get_config
import cv2
from google.colab.patches import cv2_imshow

# Global settings
NYU_DATA = False
DATASET = 'kitti'  # Let's not pick a fight with the model's dataloader

#build model
model='zoedepth'
pretrained_resource='local::/content/Depth-Anything/metric_depth/checkpoints/depth_anything_metric_depth_outdoor.pt'
config = get_config(model, "infer", DATASET)
config.pretrained_resource = pretrained_resource
model = build_model(config).to('cuda' if torch.cuda.is_available() else 'cpu')

/content/Depth-Anything/metric_depth


Params passed to Resize transform:
	width:  518
	height:  392
	resize_target:  True
	keep_aspect_ratio:  True
	ensure_multiple_of:  14
	resize_method:  minimal
Using pretrained resource local::/content/Depth-Anything/metric_depth/checkpoints/depth_anything_metric_depth_outdoor.pt
Loaded successfully


In [9]:
import math

def calculate_length(depth, roi):
    # Extract width and height from rectangle_roi
    focal_length = (math.log(66) * 29) / 0.40
    image_length = (math.log(roi) * depth) / focal_length
    image_length = round(image_length,2)

    return image_length


In [11]:
import numpy as np
import cv2
from PIL import Image
import os
import torch
from torchvision import transforms
import supervision as sv
from ultralytics import YOLO


# Initialize YOLO model and other components
modely = YOLO("/content/best.pt")
tracker = sv.ByteTrack()
box_annotator = sv.BoundingBoxAnnotator()
label_annotator = sv.LabelAnnotator()
trace_annotator = sv.TraceAnnotator()

def process_images(model, frame):
    # Convert BGR frame to RGB PIL image
    color_image = Image.fromarray(cv2.cvtColor(frame, cv2.COLOR_BGR2RGB))
    original_width, original_height = color_image.size

    # Convert PIL image to tensor and move to GPU if available
    image_tensor = transforms.ToTensor()(color_image).unsqueeze(0).to('cuda' if torch.cuda.is_available() else 'cpu')

    # Perform depth estimation
    pred = model(image_tensor, dataset="kitti")
    if isinstance(pred, dict):
        pred = pred.get('metric_depth', pred.get('out'))
    elif isinstance(pred, (list, tuple)):
        pred = pred[-1]
    pred = pred.squeeze().detach().cpu().numpy()

    # Resize depth to match the input dimensions
    resized_pred = Image.fromarray(pred).resize((original_width, original_height), Image.NEAREST)
    depth_array = np.array(resized_pred)

    # Normalize depth values and convert to CV_8U for visualization
    depth_normalized = (depth_array / 44.0) * 255.0  # Assuming the maximum depth value is 44
    depth_frame = depth_normalized.astype(np.uint8)  # Convert to CV_8U
    #depth_frame = cv2.applyColorMap(depth_8, cv2.COLOR_GRAY2BGR)
    # Perform object detection with YOLO
    results = modely(frame)[0]
    detections = sv.Detections.from_ultralytics(results)
    detections = tracker.update_with_detections(detections)

    avg_depths = []
    len = []
    for detection in detections:
      print(detection[0].astype(int))
      rectangle_roi = np.maximum(detection[0].astype(int), 0)
      # roi_depth = depth_array[rectangle_roi[1]:rectangle_roi[1]+rectangle_roi[3], rectangle_roi[0]:rectangle_roi[0]+rectangle_roi[2]]

      # avg_depth = int(np.mean(roi_depth))
      # Calculate the center coordinates of the rectangle
      # avg_depths.append(avg_depth)
      center_x = rectangle_roi[0] + ((rectangle_roi[2] - rectangle_roi[0]) // 2)
      center_y = rectangle_roi[1]+ ((rectangle_roi[3] - rectangle_roi[1]) // 2)

      # Retrieve the depth value at the center coordinates
      depth_at_center = depth_array[center_y, center_x].astype(float)
      depth_at_center = round(depth_at_center, 2)
      avg_depths.append(depth_at_center)

      length = calculate_length(depth_at_center, rectangle_roi[2]-rectangle_roi[0])
      len.append(length)

    # Annotate frame with bounding boxes and labels
    labels = [
        f"#{tracker_id} {results.names[class_id]} d={depths}m l={len}m"
        for class_id, tracker_id, depths, len
        in zip(detections.class_id, detections.tracker_id, avg_depths, len)
    ]
    annotated_frame = box_annotator.annotate(
        frame.copy(), detections=detections)
    annotated_frame = label_annotator.annotate(
        annotated_frame, detections=detections, labels=labels)


    # Annotate the frame with depth information
    # You can merge the annotated frame and depth_color here as needed

    return annotated_frame, depth_frame

def main(model, video_path, output_path1, output_path2):
    cap = cv2.VideoCapture(video_path)
    fps = cap.get(cv2.CAP_PROP_FPS)
    width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))

    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out1 = cv2.VideoWriter(output_path1, fourcc, fps, (width, height))
    out2 = cv2.VideoWriter(output_path2, fourcc, fps, (width, height))

    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break

        annotated_frame,depth_frame = process_images(model, frame)
        out1.write(annotated_frame)
        out2.write(depth_frame)
        # cv2_imshow(depth_frame)
        # cv2_imshow(annotated_frame)


    cap.release()
    out1.release()
    out2.release()
    cv2.destroyAllWindows()


video_path = "/content/drive/MyDrive/ozfish snippets/E000145_L.MP4.47323.mp4"
output1 = "/content/oz_rgb.mp4"
output2 = "/content/oz_depth.mp4"


# all_files = os.listdir(video_path)
# first_five_files = all_files[:5]
# for file_name in first_five_files:
#     vid = video_path+'/'+file_name
#     output_path1 = output1+ '/' + file_name
#     output_path2 = output2 + '/' + file_name
vid = '/content/drive/MyDrive/A000014_R.avi.9711.mp4'
main(model,vid, output1, output2)
print("done with file")


# import shutil

# shutil.move("/content/oz_depth.mp4", "/content/drive/MyDrive/depth_anything/depthmaps")
# shutil.move("/content/oz_rgb.mp4", "/content/drive/MyDrive/")


0: 384x640 3 fishs, 16.0ms
Speed: 3.0ms preprocess, 16.0ms inference, 1.8ms postprocess per image at shape (1, 3, 384, 640)
[402 406 520 450]
[492 327 565 361]
[1187  426 1261  463]

0: 384x640 5 fishs, 7.4ms
Speed: 3.5ms preprocess, 7.4ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
[408 406 521 448]
[491 326 565 361]
[1182  426 1256  463]

0: 384x640 5 fishs, 7.9ms
Speed: 4.5ms preprocess, 7.9ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
[405 404 528 450]
[490 326 565 361]
[1180  426 1252  462]

0: 384x640 4 fishs, 9.3ms
Speed: 3.1ms preprocess, 9.3ms inference, 1.3ms postprocess per image at shape (1, 3, 384, 640)
[406 402 532 449]
[488 325 565 361]
[1176  426 1247  462]
[896 452 961 503]

0: 384x640 4 fishs, 12.6ms
Speed: 3.5ms preprocess, 12.6ms inference, 2.0ms postprocess per image at shape (1, 3, 384, 640)
[412 400 537 447]
[488 325 565 361]
[1172  427 1241  462]
[897 451 962 502]

0: 384x640 4 fishs, 8.9ms
Speed: 3.1ms preprocess, 8.9

In [ ]:

import shutil

shutil.move("/content/oz_depth.mp4", "/content/drive/MyDrive/")
shutil.move("/content/oz_rgb.mp4", "/content/drive/MyDrive/")

'/content/drive/MyDrive/oz_rgb.mp4'